In [10]:
import os, json, time, pdfplumber, re, locale, datetime
import pandas as pd
from tqdm import tqdm
from tqdm.notebook import tqdm as ntqdm

In [11]:
locale.setlocale(locale.LC_ALL, 'id_ID.utf8')

'id_ID.utf8'

In [2]:
filename = 'KIB - 02.pdf'

# Get Page which isn't a first page

No result. No need to run again

In [8]:
with pdfplumber.open(filename) as pdf:
    pbar = ntqdm(total=len(pdf.pages))

    for i in range(len(pdf.pages)):
        page = pdf.pages[i]
        text = page.extract_text()
        if 'kementerian pendidikan, kebudayaan, riset' not in text.lower():
            print(i+1)
        pbar.update(1)
    pbar.close()

  0%|          | 0/614 [00:00<?, ?it/s]

# Pull Data

In [29]:
li = []

with pdfplumber.open(filename) as pdf:
    totl = len(pdf.pages)
    #totl = 5
    pbar = ntqdm(total=totl)

    for i in range(totl):
        temp_dict = {}
        page = pdf.pages[i]
        text = page.extract_text()
        
        if 'kementerian pendidikan, kebudayaan, riset' in text.lower():
            for line in text.split('\n'):
                text_lower = line.lower()
                if 'a.nup' in text_lower:
                    temp_dict['nup'] = line.split(':')[-1].strip()

                if 'b.kode barang' in text_lower:
                    temp_dict['kode_barang'] = line.split(':')[-1].strip()

                if 'bidang' in text_lower:
                    temp_dict['bidang'] = line.split(':')[-1].strip()

                if ('kelompok' in text_lower) and ('sub' not in text_lower):
                    temp_dict['kelompok'] = line.split(':')[-1].strip()
                
                if ('sub kelompok' in text_lower) and ('sub-sub' not in text_lower):
                    temp_dict['sub_kelompok'] = line.split(':')[-1].strip()

                if ('sub-sub kelompok' in text_lower) and ('sub-sub-sub' not in text_lower):
                    temp_dict['sub_sub_kelompok'] = line.split(':')[-1].strip()

                if ('sub-sub-sub kelompok' in text_lower):
                    temp_dict['sub_sub_sub_kelompok'] = line.split(':')[-1].strip()

                if ('nama uakpb' in text_lower):
                    temp_dict['nama_uakpb'] = line.split(':')[-1].strip()

                if ('kode uakpb' in text_lower):
                    temp_dict['kode_uakpb'] = line.split(':')[-1].strip()

            left = page.crop((0, 0.3 * float(page.height), 0.5 * float(page.width), float(page.height)))
            right = page.crop((0.5 * float(page.width), 0.3 * float(page.height), float(page.width), float(page.height)))

            left_text = left.extract_text()
            right_text = right.extract_text()

            luas_bangunan_text = ''
            jumlah_lantai_text = ''
            type_text = ''
            th_sls_dibangun_text = ''
            no_imb_text = ''
            tgl_imb_text = ''
            letak_bangunan_text = ''
            propinsi_text = ''
            kota_text = ''
            kecamatan_text = ''
            kelurahan_text = ''
            jalan_text = ''
            rt_text = ''
            no_kib_tanah_text = ''
            nama_unit_text = ''
            alamat_unit_text = ''
            disetujui_tgl_text = ''
            disetujui_nama_pejabat_text = ''
            disetujui_jbt_struktural_text = ''
            jalan_start = False
            nama_pejabat_start = False
            jbt_struktural_start = False

            for line in left_text.split('\n'):
                text_lower = line.lower()

                if 'luas bangunan' in text_lower:
                    luas_bangunan_text += line.split(':')[-1].strip() + ' '
                
                if 'jumlah lantai' in text_lower:
                    temp_dict['luas_bangunan'] = re.sub(' +', ' ', luas_bangunan_text).strip()
                    temp_dict['luas_bangunan_nilai'] = float(re.sub(' +', ' ', luas_bangunan_text).replace('M2', '').strip())

                    jumlah_lantai_text += line.split(':')[-1].strip() + ' '

                if 'type' in text_lower:
                    temp_dict['jumlah_lantai'] = re.sub(' +', ' ', jumlah_lantai_text).strip()
                    temp_dict['jumlah_lantai_nilai'] = float(re.sub(' +', ' ', jumlah_lantai_text).strip().replace('Lantai', '').strip())
                    type_text += line.split(':')[-1].strip() + ' '

                if 'th, sls dibangun' in text_lower:
                    temp_dict['type'] = re.sub(' +', ' ', type_text).strip()
                    th_sls_dibangun_text += line.split(':')[-1].strip() + ' '

                if 'no imb' in text_lower:
                    temp_dict['th_sls_dibangun'] = re.sub(' +', ' ', th_sls_dibangun_text).strip()
                    no_imb_text += line.split(':')[-1].strip() + ' '

                if 'tgl imb' in text_lower:
                    temp_dict['no_imb'] = re.sub(' +', ' ', no_imb_text).strip()
                    tgl_imb_text += line.split(':')[-1].strip() + ' '

                if 'letak bangunan' in text_lower:
                    temp_dict['tgl_imb'] = re.sub(' +', ' ', tgl_imb_text).strip()
                    if tgl_imb_text: temp_dict['tgl_imb_tgl'] = datetime.datetime.strptime(tgl_imb_text.strip(), '%d %B %Y')
                    else: temp_dict['tgl_imb_tgl'] = None
                    letak_bangunan_text += line.split(':')[-1].strip() + ' '

                if 'propinsi' in text_lower:
                    temp_dict['letak_bangunan'] = re.sub(' +', ' ', letak_bangunan_text).strip()
                    propinsi_text += line.split(':')[-1].strip() + ' '

                if 'kota/kab' in text_lower:
                    temp_dict['propinsi'] = re.sub(' +', ' ', propinsi_text).strip()
                    kota_text += line.split(':')[-1].strip() + ' '

                if 'kecamatan' in text_lower:
                    temp_dict['kota_kab'] = re.sub(' +', ' ', kota_text).strip()
                    kecamatan_text += line.split(':')[-1].strip() + ' '

                if 'kelurahan/desa' in text_lower:
                    temp_dict['kecamatan'] = re.sub(' +', ' ', kecamatan_text).strip()
                    kelurahan_text += line.split(':')[-1].strip() + ' '

                if 'jalan' in text_lower:
                    temp_dict['kelurahan_desa'] = re.sub(' +', ' ', kelurahan_text).strip()
                    jalan_start = True

                if 'rt/rw/rk' in text_lower:
                    temp_dict['jalan'] = re.sub(' +', ' ', jalan_text).strip()
                    rt_text += line.split(':')[-1].strip() + ' '
                    jalan_start = False

                if jalan_start:
                    jalan_text += line.split(':')[-1] + ' '

                if 'no kib tanah' in text_lower:
                    temp_dict['rt_rw_rk'] = re.sub(' +', ' ', rt_text).strip()
                    no_kib_tanah_text += line.split(':')[-1].strip() + ' '

                if 'nama unit' in text_lower:
                    temp_dict['no_kib_tanah'] = re.sub(' +', ' ', no_kib_tanah_text).strip()
                    nama_unit_text += line.split(':')[-1].strip() + ' '

                if 'alamat' in text_lower:
                    temp_dict['nama_unit'] = re.sub(' +', ' ', nama_unit_text).strip()
                    alamat_unit_text += line.split(':')[-1].strip() + ' '

                if 'disetujui tgl.' in text_lower:
                    temp_dict['alamat'] = re.sub(' +', ' ', alamat_unit_text).strip()
                    disetujui_tgl_text += line.split(':')[-1].strip() + ' '

                if 'nama pejabat.' in text_lower:
                    temp_dict['disetujui_tgl'] = re.sub(' +', ' ', disetujui_tgl_text).strip()
                    if disetujui_tgl_text.strip(): temp_dict['disetujui_tgl_tgl'] = datetime.datetime.strptime(disetujui_tgl_text.strip(), '%d %B %Y')
                    else: temp_dict['disetujui_tgl_tgl'] = None
                    nama_pejabat_start = True

                if 'jbt struktural.' in text_lower:
                    temp_dict['disetujui_nama_pejabat'] = re.sub(' +', ' ', disetujui_nama_pejabat_text).strip()
                    jbt_struktural_start = True
                    nama_pejabat_start = False

                if nama_pejabat_start:
                    disetujui_nama_pejabat_text += line.split(':')[-1] + ' '

                if 'tanda tangan dan stempel' in text_lower:
                    temp_dict['disetujui_jbt_struktural'] = re.sub(' +', ' ', disetujui_jbt_struktural_text).strip()
                    jbt_struktural_start = False

                if jbt_struktural_start:
                    disetujui_jbt_struktural_text += line.split(':')[-1] + ' '

            cara_perolehan_text = ''
            dari_text = ''
            tgl_perolehan_text = ''
            kondisi_perolehan_text = ''
            harga_perolehan_text = ''
            dasar_harga_text = ''
            sumber_dana_text = ''
            no_text = ''
            tgl_text = ''
            nilai_buku_text = ''
            nilai_wajar_text = ''
            njop_text = ''
            status_penggunaan_text = ''
            digunakan_text = ''
            diisi_tgl_text = ''
            diisi_nama_pejabat_text = ''
            diisi_jabatan_text = ''
            digunakan_start = False
            nama_pejabat_start = False
            jbt_struktural_start = False

            for line in right_text.split('\n'):
                #print(line)
                text_lower = line.lower()

                if 'cara perolehan' in text_lower:
                    cara_perolehan_text += line.split(':')[-1].strip() + ' '
                
                if '2. dari' in text_lower:
                    temp_dict['cara_perolehan'] = re.sub(' +', ' ', cara_perolehan_text).strip()
                    dari_text += line.split(':')[-1].strip() + ' '

                if '3. tgl perolehan' in text_lower:
                    temp_dict['dari'] = re.sub(' +', ' ', dari_text).strip()
                    tgl_perolehan_text += line.split(':')[-1].strip() + ' '

                if 'kondisi perolehan' in text_lower:
                    temp_dict['tgl_perolehan'] = re.sub(' +', ' ', tgl_perolehan_text).strip()
                    if tgl_perolehan_text.strip(): temp_dict['tgl_perolehan_tgl'] = datetime.datetime.strptime(tgl_perolehan_text.strip(), '%d %B %Y')
                    else: temp_dict['tgl_perolehan_tgl'] = None
                    kondisi_perolehan_text += line.split(':')[-1].strip() + ' '

                if 'harga perolehan' in text_lower:
                    temp_dict['kondisi_perolehan'] = re.sub(' +', ' ', kondisi_perolehan_text).strip()
                    harga_perolehan_text += line.split(':')[-1].strip() + ' '

                if 'dasar harga' in text_lower:
                    temp_dict['harga_perolehan'] = re.sub(' +', ' ', harga_perolehan_text).strip()
                    temp_dict['harga_perolehan_nilai'] = float(re.sub(' +', ' ', harga_perolehan_text).strip().split()[-1].replace('.', '').replace(',', ''))
                    dasar_harga_text += line.split(':')[-1].strip() + ' '

                if 'sumber dana' in text_lower:
                    temp_dict['dasar_harga'] = re.sub(' +', ' ', dasar_harga_text).strip()
                    sumber_dana_text += line.split(':')[-1].strip() + ' '

                if 'no.' in text_lower:
                    temp_dict['sumber_dana'] = re.sub(' +', ' ', sumber_dana_text).strip()
                    no_text += line.split(':')[-1].strip() + ' '

                if 'tgl.' in text_lower:
                    temp_dict['no'] = re.sub(' +', ' ', no_text).strip()
                    tgl_text += line.split(':')[-1].strip() + ' '

                if 'nilai buku' in text_lower:
                    temp_dict['tgl'] = re.sub(' +', ' ', tgl_text).strip()
                    nilai_buku_text += line.split(':')[-1].strip() + ' '

                if 'nilai wajar' in text_lower:
                    temp_dict['nilai_buku'] = re.sub(' +', ' ', nilai_buku_text).strip()
                    temp_dict['nilai_buku_nilai'] = float(re.sub(' +', ' ', nilai_buku_text).strip().split()[-1].replace('.', '').replace(',', ''))
                    nilai_wajar_text += line.split(':')[-1].strip() + ' '

                if 'njop' in text_lower:
                    temp_dict['nilai_wajar'] = re.sub(' +', ' ', nilai_wajar_text).strip()
                    njop_text += line.split(':')[-1].strip() + ' '

                if 'status penggunaan' in text_lower:
                    temp_dict['njop'] = re.sub(' +', ' ', njop_text).strip()
                    status_penggunaan_text += line.split(':')[-1].strip() + ' '

                if '2. digunakan' in text_lower:
                    temp_dict['status_penggunaan'] = re.sub(' +', ' ', status_penggunaan_text).strip()
                    digunakan_start = True

                if 'diisi tgl.' in text_lower:
                    temp_dict['digunakan'] = re.sub(' +', ' ', digunakan_text).strip()
                    diisi_tgl_text += line.split(':')[-1].strip() + ' '
                    digunakan_start = False

                if digunakan_start:
                    digunakan_text += line.split(':')[-1] + ' '

                if 'nama pejabat.' in text_lower:
                    temp_dict['diisi_tgl'] = re.sub(' +', ' ', diisi_tgl_text).strip()
                    if diisi_tgl_text.strip(): temp_dict['diisi_tgl_tgl'] = datetime.datetime.strptime(diisi_tgl_text.strip(), '%d %B %Y')
                    else: temp_dict['diisi_tgl_tgl'] = None
                    nama_pejabat_start = True

                if 'jabatan.' in text_lower:
                    temp_dict['diisi_nama_pejabat'] = re.sub(' +', ' ', diisi_nama_pejabat_text).strip()
                    jbt_struktural_start = True
                    nama_pejabat_start = False

                if nama_pejabat_start:
                    diisi_nama_pejabat_text += line.split(':')[-1] + ' '

                if 'tanda tangan dan stempel' in text_lower:
                    temp_dict['diisi_jabatan'] = re.sub(' +', ' ', diisi_jabatan_text).strip()
                    jbt_struktural_start = False

                if jbt_struktural_start:
                    diisi_jabatan_text += line.split(':')[-1] + ' '

        pbar.update(1)
        li.append(temp_dict)
    pbar.close()

df = pd.DataFrame(li)


  0%|          | 0/614 [00:00<?, ?it/s]

In [30]:
df

,nup,kode_barang,bidang,kelompok,sub_kelompok,sub_sub_kelompok,sub_sub_sub_kelompok,nama_uakpb,kode_uakpb,luas_bangunan,...,nilai_buku,nilai_buku_nilai,nilai_wajar,njop,status_penggunaan,digunakan,diisi_tgl,diisi_tgl_tgl,diisi_nama_pejabat,diisi_jabatan
0,1,02.01.01.01.001,GEDUNG DAN BANGUNAN,BANGUNAN GEDUNG,BANGUNAN GEDUNG TEMPAT KERJA,BANGUNAN GEDUNG KANTOR,BANGUNAN GEDUNG KANTOR PERMANEN,Fakultas Kedokteran,023170500677512007KD,8185 M2,...,Rp. 13.394.054.267,1.339405e+10,,,Digunakan Sendiri Untuk Dinas Jabatan,OPERASIONAL PENDIDIKAN FAKULTAS KEDOKTERAN UNI...,13 Juli 2023,2023-07-13,"Tatipatto, Naomi Yuanita, SH",Pengelola Barang Milik Negara
1,1,02.01.01.01.001,GEDUNG DAN BANGUNAN,BANGUNAN GEDUNG,BANGUNAN GEDUNG TEMPAT KERJA,BANGUNAN GEDUNG KANTOR,BANGUNAN GEDUNG KANTOR PERMANEN,Fakultas Teknologi Pertanian,023170500677512010KD,2930 M2,...,Rp. 136.222.490.115,1.362225e+11,,,Digunakan Sendiri Untuk Dinas Jabatan,PERKANTORAN DAN PERKULIAHAN,14 Juli 2023,2023-07-14,,
2,2,02.01.01.01.001,GEDUNG DAN BANGUNAN,BANGUNAN GEDUNG,BANGUNAN GEDUNG TEMPAT KERJA,BANGUNAN GEDUNG KANTOR,BANGUNAN GEDUNG KANTOR PERMANEN,Fakultas Teknologi Pertanian,023170500677512010KD,1096 M2,...,Rp. 19.157.494.903,1.915749e+10,,,Digunakan Sendiri Untuk Dinas Jabatan,PERKANTORAN DAN PERKULIAHAN,14 Juli 2023,2023-07-14,,
3,1,02.01.01.12.999,GEDUNG DAN BANGUNAN,BANGUNAN GEDUNG,BANGUNAN GEDUNG TEMPAT KERJA,BANGUNAN GEDUNG PERTOKOAN/KOPERASI/PASAR,BANGUNAN GEDUNG PERTOKOAN/KOPERASI/PASAR LAINNYA,Fakultas Teknologi Pertanian,023170500677512010KD,334 M2,...,Rp. 1.649.775.203,1.649775e+09,,,Digunakan Sendiri Untuk Dinas Jabatan,KANTIN - LKM,14 Juli 2023,2023-07-14,,
4,1,02.01.01.14.001,GEDUNG DAN BANGUNAN,BANGUNAN GEDUNG,BANGUNAN GEDUNG TEMPAT KERJA,BANGUNAN GEDUNG GARASI/POOL,GEDUNG GARASI/POOL PERMANEN,Fakultas Teknologi Pertanian,023170500677512010KD,87 M2,...,Rp. 151.200.000,1.512000e+08,,,Digunakan Sendiri Untuk Dinas Jabatan,GARASI MOBIL DINAS,14 Juli 2023,2023-07-14,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,2,02.01.01.34.001,Staff Umum dan Aset,BANGUNAN GEDUNG,BANGUNAN GEDUNG TEMPAT KERJA,TAMAN,TAMAN PERMANEN,Fakultas Teknik,023170500677512006KD,366 M2,...,Rp. 64.999.788,6.499979e+07,,,Pemanfaatan Lainnya sesuai Ketentuan,Taman,14 Juli 2023,2023-07-14,"Sugiantoro, SE",Staff Umum dan Aset
610,3,02.01.01.34.001,Staff Umum dan Aset,BANGUNAN GEDUNG,BANGUNAN GEDUNG TEMPAT KERJA,TAMAN,TAMAN PERMANEN,Fakultas Teknik,023170500677512006KD,10 M2,...,Rp. 137.592.818,1.375928e+08,,,Pemanfaatan Lainnya sesuai Ketentuan,Taman,14 Juli 2023,2023-07-14,"Sugiantoro, SE",Staff Umum dan Aset
611,4,02.01.01.34.001,Staff Umum dan Aset,BANGUNAN GEDUNG,BANGUNAN GEDUNG TEMPAT KERJA,TAMAN,TAMAN PERMANEN,Fakultas Teknik,023170500677512006KD,78 M2,...,Rp. 156.314.520,1.563145e+08,,,Pemanfaatan Lainnya sesuai Ketentuan,Taman,14 Juli 2023,2023-07-14,"Sugiantoro, SE",Staff Umum dan Aset
612,1,02.01.01.29.002,GEDUNG DAN BANGUNAN,BANGUNAN GEDUNG,BANGUNAN GEDUNG TEMPAT KERJA,BANGUNAN PETERNAKAN/PERIKANAN,BANGUNAN KOLAM/BAK IKAN,Fakultas Kedokteran,023170500677512007KD,10 M2,...,Rp. 39.600.000,3.960000e+07,,,Pemanfaatan Lainnya sesuai Ketentuan,Kolam Ikan,13 Juli 2023,2023-07-13,"Tatipatto, Naomi Yuanita, SH",Pengelola Barang Milik Negara


In [31]:
df.to_excel(filename.replace('.pdf', '.xlsx'), index=False)

In [24]:
df.iloc[0, :]

nup                                                                         1
kode_barang                                                   02.01.01.01.001
bidang                                                    GEDUNG DAN BANGUNAN
kelompok                                                      BANGUNAN GEDUNG
sub_kelompok                                     BANGUNAN GEDUNG TEMPAT KERJA
sub_sub_kelompok                                       BANGUNAN GEDUNG KANTOR
sub_sub_sub_kelompok                          BANGUNAN GEDUNG KANTOR PERMANEN
nama_uakpb                                                Fakultas Kedokteran
kode_uakpb                                               023170500677512007KD
luas_bangunan                                                         8185 M2
jumlah_lantai                                                        5 Lantai
type                                                                        -
th_sls_dibangun                                                 